## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [1]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

# Testing to make sure TensorFlow GPU is working
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17812529919537992452
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3230007296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13479776756620670816
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


### Load data, shuffle data, normalize data, split data into training and validation

In [2]:
from sklearn.utils import shuffle

# Load the data
X_train = np.load("X_train_3.dat")
y_train = np.load("y_train_3.dat")


X_train, y_train = shuffle(X_train, y_train, random_state=42)
# reshape so in form for CNN-Keras
#X_train = X_train.reshape(X_train.shape[0], 174, 124, 1)

# Normalize the data
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

# split data into validation set
training_set_size = int(X_train.shape[0] * .80)
X_train_validation = X_train[training_set_size:, :,:,:]
y_train_validation = y_train[training_set_size:, :]
y_train = y_train[:training_set_size,:]
X_train = X_train[:training_set_size, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(9000, 124, 174, 3)
(9000, 10)
Validation x train set:(1800, 124, 174, 3)
X train set:(7200, 124, 174, 3)
Validation y train set:(1800, 10)
Y train set:(7200, 10)


## Build the CNN network

In [3]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (7,7), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(32, (7,7), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv2D(64, (7, 7), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Conv2D(64, (7, 7), strides=(1,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.75))
model.add(Dense(10, activation='softmax'))

## Begin training the model

In [ ]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model_3splits_200epochs_midLR", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

adam = Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(adam, 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=200 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 7200 samples, validate on 1800 samples
Epoch 1/200
7200/7200 [==============================] - 26s 4ms/step - loss: 2.4823 - acc: 0.2112 - val_loss: 1.8573 - val_acc: 0.3739
Epoch 2/200
7200/7200 [==============================] - 24s 3ms/step - loss: 1.9771 - acc: 0.2887 - val_loss: 1.8455 - val_acc: 0.3400
Epoch 3/200
7200/7200 [==============================] - 24s 3ms/step - loss: 1.8120 - acc: 0.3421 - val_loss: 1.7084 - val_acc: 0.4056
Epoch 4/200
6752/7200 [===========================>..] - ETA: 1s - loss: 1.7581 - acc: 0.3556

In [13]:
model.save("models/trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]


### Reset Keras

In [53]:
from keras import backend as K
K.clear_session()

In [ ]:
### Run a NN without Convolutions (ANN)

In [9]:
ann_model = Sequential()

ann_model.add(Flatten())
ann_model.add(Dense(512, activation = 'relu', input_shape = X_train.shape[1:]))
#ann_model.add(Dropout(0.5))

ann_model.add(Dense(256, activation = 'relu'))
#ann_model.add(Dropout(0.5))

ann_model.add(Dense(128, activation = 'relu'))
#ann_model.add(Dropout(0.5))

ann_model.add(Dense(64, activation = 'relu'))
#ann_model.add(Dropout(0.5))

ann_model.add(Dense(10, activation = 'softmax'))

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model_2splits_50epochs_highLR", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

#adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)

# Using custom adam is horrifically bad for ANN...
ann_model.compile('adam', 'categorical_crossentropy',metrics=['accuracy'])

ann_model.fit(X_train, y_train, batch_size=32, epochs=50 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 10800 samples, validate on 2700 samples
Epoch 1/50
10800/10800 [==============================] - 9s 808us/step - loss: 2.2177 - acc: 0.2615 - val_loss: 1.8678 - val_acc: 0.3263
Epoch 2/50
10800/10800 [==============================] - 8s 762us/step - loss: 1.7888 - acc: 0.3276 - val_loss: 1.7145 - val_acc: 0.3489
Epoch 3/50
10800/10800 [==============================] - 8s 763us/step - loss: 1.7394 - acc: 0.3336 - val_loss: 1.7266 - val_acc: 0.3441
Epoch 4/50
10800/10800 [==============================] - 8s 762us/step - loss: 1.7063 - acc: 0.3481 - val_loss: 1.6665 - val_acc: 0.3526
Epoch 5/50
10800/10800 [==============================] - 8s 765us/step - loss: 1.6637 - acc: 0.3619 - val_loss: 1.6435 - val_acc: 0.3663
Epoch 6/50
10800/10800 [==============================] - 8s 769us/step - loss: 1.6464 - acc: 0.3694 - val_loss: 1.6415 - val_acc: 0.3593
Epoch 7/50
10800/10800 [==============================] - 8s 771us/step - loss: 1.6206 - acc: 0.3740 - val_loss: 1.6603 - v

In [11]:
ann_model.fit(X_train, y_train, batch_size=32, epochs=50 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 10800 samples, validate on 2700 samples
Epoch 1/50
10800/10800 [==============================] - 8s 758us/step - loss: 0.6543 - acc: 0.7602 - val_loss: 1.6736 - val_acc: 0.5515
Epoch 2/50
10800/10800 [==============================] - 8s 771us/step - loss: 0.6608 - acc: 0.7577 - val_loss: 1.4058 - val_acc: 0.6122
Epoch 3/50
10800/10800 [==============================] - 8s 757us/step - loss: 0.6175 - acc: 0.7681 - val_loss: 1.3964 - val_acc: 0.6078
Epoch 4/50
10800/10800 [==============================] - 8s 758us/step - loss: 0.6041 - acc: 0.7754 - val_loss: 1.3546 - val_acc: 0.6093
Epoch 5/50
10800/10800 [==============================] - 8s 763us/step - loss: 0.5754 - acc: 0.7869 - val_loss: 1.6289 - val_acc: 0.5681
Epoch 6/50
10800/10800 [==============================] - 8s 762us/step - loss: 0.6426 - acc: 0.7673 - val_loss: 1.4995 - val_acc: 0.5974
Epoch 7/50
10800/10800 [==============================] - 8s 763us/step - loss: 0.5832 - acc: 0.7793 - val_loss: 1.2997 - v